# Venture Funding with Deep Learning

You work as a risk management associate at Alphabet Soup, a venture capital firm. Alphabet Soup’s business team receives many funding applications from startups every day. This team has asked you to help them create a model that predicts whether applicants will be successful if funded by Alphabet Soup.

The business team has given you a CSV containing more than 34,000 organizations that have received funding from Alphabet Soup over the years. With your knowledge of machine learning and neural networks, you decide to use the features in the provided dataset to create a binary classifier model that will predict whether an applicant will become a successful business. The CSV file contains a variety of information about these businesses, including whether or not they ultimately became successful.

## Instructions:

The steps for this challenge are broken out into the following sections:
* Prepare the data for use on a neural network model.
* Compile and evaluate a binary classification model using a neural network.
* Optimize the neural network model.

### Prepare the Data for Use on a Neural Network Model 

Using your knowledge of Pandas and scikit-learn’s `StandardScaler()`, preprocess the dataset so that you can use it to compile and evaluate the neural network model later.

Open the starter code file, and complete the following data preparation steps:

1. Read the `applicants_data.csv` file into a Pandas DataFrame. Review the DataFrame, looking for categorical variables that will need to be encoded, as well as columns that could eventually define your features and target variables.   

2. Drop the “EIN” (Employer Identification Number) and “NAME” columns from the DataFrame, because they are not relevant to the binary classification model.
 
3. Encode the dataset’s categorical variables using `OneHotEncoder`, and then place the encoded variables into a new DataFrame.

4. Add the original DataFrame’s numerical variables to the DataFrame containing the encoded variables.

> **Note** To complete this step, you will employ the Pandas `concat()` function that was introduced earlier in this course. 

5. Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “IS_SUCCESSFUL”. The remaining columns should define the features dataset. 

6. Split the features and target sets into training and testing datasets.

7. Use scikit-learn's `StandardScaler` to scale the features data.

### Compile and Evaluate a Binary Classification Model Using a Neural Network

Use your knowledge of TensorFlow to design a binary classification deep neural network model. This model should use the dataset’s features to predict whether an Alphabet Soup&ndash;funded startup will be successful based on the features in the dataset. Consider the number of inputs before determining the number of layers that your model will contain or the number of neurons on each layer. Then, compile and fit your model. Finally, evaluate your binary classification model to calculate the model’s loss and accuracy. 
 
To do so, complete the following steps:

1. Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.

2. Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.

> **Hint** When fitting the model, start with a small number of epochs, such as 20, 50, or 100.

3. Evaluate the model using the test data to determine the model’s loss and accuracy.

4. Save and export your model to an HDF5 file, and name the file `AlphabetSoup.h5`. 

### Optimize the Neural Network Model

Using your knowledge of TensorFlow and Keras, optimize your model to improve the model's accuracy. Even if you do not successfully achieve a better accuracy, you'll need to demonstrate at least two attempts to optimize the model. You can include these attempts in your existing notebook. Or, you can make copies of the starter notebook in the same folder, rename them, and code each model optimization in a new notebook. 

> **Note** You will not lose points if your model does not achieve a high accuracy, as long as you make at least two attempts to optimize the model.

To do so, complete the following steps:

1. Define at least three new deep neural network models (the original plus 2 optimization attempts). With each, try to improve on your first model’s predictive accuracy.

> **Rewind** Recall that perfect accuracy has a value of 1, so accuracy improves as its value moves closer to 1. To optimize your model for a predictive accuracy as close to 1 as possible, you can use any or all of the following techniques:
> * Adjust the input data by dropping different features columns to ensure that no variables or outliers confuse the model.
> * Add more neurons (nodes) to a hidden layer.
> * Add more hidden layers.
> * Use different activation functions for the hidden layers.
> * Add to or reduce the number of epochs in the training regimen.

2. After finishing your models, display the accuracy scores achieved by each model, and compare the results.

3. Save each of your models as an HDF5 file.

In [236]:
# Imports
import pandas as pd
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder

pd.set_option('display.max_columns', None)

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `applicants_data.csv` file into a Pandas DataFrame. Review the DataFrame, looking for categorical variables that will need to be encoded, as well as columns that could eventually define your features and target variables.  


In [237]:
# Read the applicants_data.csv file from the Resources folder into a Pandas DataFrame
applicant_data_df = pd.read_csv(Path('Resources/applicants_data.csv'))

# Review the DataFrame
print(applicant_data_df.shape)
applicant_data_df.tail()

(34299, 12)


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
34294,996009318,THE LIONS CLUB OF HONOLULU KAMEHAMEHA,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,996010315,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,996012607,PTA HAWAII CONGRESS,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,996015768,AMERICAN FEDERATION OF GOVERNMENT EMPLOYEES LO...,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1
34298,996086871,WATERHOUSE CHARITABLE TR,T3,Independent,C1000,Preservation,Co-operative,1,1M-5M,N,36500179,0


In [238]:
# Review the data types associated with the columns
applicant_data_df.dtypes

EIN                        int64
NAME                      object
APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [239]:
# Checking the object type data
applicant_data_df.describe(include='object')

# 'SPECIAL_CONSIDERATIONS' only has two unique values, so might want to think 
#  about dropping an encoded value later, to avoid the 'Dummy Variable Trap'.

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,SPECIAL_CONSIDERATIONS
count,34299,34299,34299,34299,34299,34299,34299,34299
unique,19568,17,6,71,5,4,9,2
top,PARENT BOOSTER USA INC,T3,Independent,C1000,Preservation,Trust,0,N
freq,1260,27037,18480,17326,28095,23515,24388,34272


In [240]:
# Checking the numbrer type data
applicant_data_df.describe(include='int64')

# hmm, 'STATUS' and 'IS_SUCCESSFUL' look like they should actually be categorical variables, 
#  since the only values are 0 and 1.
# Ah, encoded categorical variables are cast to 0 and 1 int64 anyway, so it's fine that these 
#  two variables are int64s. 

,EIN,STATUS,ASK_AMT,IS_SUCCESSFUL
count,3.429900e+04,34299.000000,3.429900e+04,34299.000000
mean,5.191852e+08,0.999854,2.769199e+06,0.532406
std,2.451472e+08,0.012073,8.713045e+07,0.498956
min,1.052060e+07,0.000000,5.000000e+03,0.000000
25%,2.748482e+08,1.000000,5.000000e+03,0.000000
50%,4.656317e+08,1.000000,5.000000e+03,1.000000
75%,7.526117e+08,1.000000,7.742000e+03,1.000000
max,9.960869e+08,1.000000,8.597806e+09,1.000000


### Step 2: Drop the “EIN” (Employer Identification Number) and “NAME” columns from the DataFrame, because they are not relevant to the binary classification model.

In [241]:
# Drop the 'EIN' and 'NAME' columns from the DataFrame
applicant_data_df = applicant_data_df.drop(columns=['EIN', 'NAME'])

# Review the DataFrame
print(applicant_data_df.shape)
applicant_data_df.head()

(34299, 10)


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


### Step 3: Encode the dataset’s categorical variables using `OneHotEncoder`, and then place the encoded variables into a new DataFrame.

In [242]:
# Create a list of categorical variables 
categorical_variables = list(applicant_data_df.dtypes[applicant_data_df.dtypes == 'object'].index)

# Display the categorical variables list
categorical_variables

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [243]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

In [244]:
# Encode the categorcal variables using OneHotEncoder
encoded_data = enc.fit_transform(applicant_data_df[categorical_variables])

In [245]:
# Create a DataFrame with the encoded variables
encoded_df = pd.DataFrame(encoded_data, columns=enc.get_feature_names(categorical_variables))

# Review the DataFrame
encoded_df.head()

,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,APPLICATION_TYPE_T9,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,CLASSIFICATION_C0,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C1230,CLASSIFICATION_C1234,CLASSIFICATION_C1235,CLASSIFICATION_C1236,CLASSIFICATION_C1237,CLASSIFICATION_C1238,CLASSIFICATION_C1240,CLASSIFICATION_C1245,CLASSIFICATION_C1246,CLASSIFICATION_C1248,CLASSIFICATION_C1250,CLASSIFICATION_C1256,CLASSIFICATION_C1257,CLASSIFICATION_C1260,CLASSIFICATION_C1267,CLASSIFICATION_C1270,CLASSIFICATION_C1278,CLASSIFICATION_C1280,CLASSIFICATION_C1283,CLASSIFICATION_C1300,CLASSIFICATION_C1370,CLASSIFICATION_C1400,CLASSIFICATION_C1500,CLASSIFICATION_C1570,CLASSIFICATION_C1580,CLASSIFICATION_C1600,CLASSIFICATION_C1700,CLASSIFICATION_C1720,CLASSIFICATION_C1728,CLASSIFICATION_C1732,CLASSIFICATION_C1800,CLASSIFICATION_C1820,CLASSIFICATION_C1900,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C2150,CLASSIFICATION_C2170,CLASSIFICATION_C2190,CLASSIFICATION_C2300,CLASSIFICATION_C2380,CLASSIFICATION_C2400,CLASSIFICATION_C2500,CLASSIFICATION_C2561,CLASSIFICATION_C2570,CLASSIFICATION_C2600,CLASSIFICATION_C2700,CLASSIFICATION_C2710,CLASSIFICATION_C2800,CLASSIFICATION_C3000,CLASSIFICATION_C3200,CLASSIFICATION_C3700,CLASSIFICATION_C4000,CLASSIFICATION_C4100,CLASSIFICATION_C4120,CLASSIFICATION_C4200,CLASSIFICATION_C4500,CLASSIFICATION_C5000,CLASSIFICATION_C5200,CLASSIFICATION_C6000,CLASSIFICATION_C6100,CLASSIFICATION_C7000,CLASSIFICATION_C7100,CLASSIFICATION_C7120,CLASSIFICATION_C7200,CLASSIFICATION_C7210,CLASSIFICATION_C8000,CLASSIFICATION_C8200,CLASSIFICATION_C8210,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

### Step 4: Add the original DataFrame’s numerical variables to the DataFrame containing the encoded variables.

> **Note** To complete this step, you will employ the Pandas `concat()` function that was introduced earlier in this course. 

In [246]:
# Add the numerical variables from the original DataFrame to the one-hot encoding DataFrame
numerical_variables_df = applicant_data_df.select_dtypes(include='int64')
processed_df = pd.concat([numerical_variables_df, encoded_df], axis=1)

# Review the Dataframe
print(f"Shape: {processed_df.shape}\n")
print(f"dtypes:\n{processed_df.dtypes}")

Shape: (34299, 117)

dtypes:
STATUS                        int64
ASK_AMT                       int64
IS_SUCCESSFUL                 int64
APPLICATION_TYPE_T10        float64
APPLICATION_TYPE_T12        float64
                             ...   
INCOME_AMT_25000-99999      float64
INCOME_AMT_50M+             float64
INCOME_AMT_5M-10M           float64
SPECIAL_CONSIDERATIONS_N    float64
SPECIAL_CONSIDERATIONS_Y    float64
Length: 117, dtype: object


### Step 5: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “IS_SUCCESSFUL”. The remaining columns should define the features dataset. 



In [247]:
# Define the target set y using the IS_SUCCESSFUL column
y = processed_df['IS_SUCCESSFUL']

# Display a sample of y
y[:5]

0    1
1    1
2    0
3    1
4    1
Name: IS_SUCCESSFUL, dtype: int64

In [248]:
# Define features set X by selecting all columns but IS_SUCCESSFUL
X = processed_df.drop(columns=['IS_SUCCESSFUL'])

# Review the features DataFrame
print(f"Shape: {X.shape}\n")

Shape: (34299, 116)



### Step 6: Split the features and target sets into training and testing datasets.


In [249]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 7: Use scikit-learn's `StandardScaler` to scale the features data.

In [250]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

---

## Compile and Evaluate a Binary Classification Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [251]:
# Define the the number of inputs (features) to the model
number_input_features = X.shape[1]

# Review the number of features
number_input_features

116

In [252]:
# Define the number of neurons in the output layer
number_output_neurons = 1

In [253]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 =  (number_input_features + 1) // 2

# Review the number hidden nodes in the first layer
hidden_nodes_layer1

58

In [254]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 = (hidden_nodes_layer1 + 1) // 2

# Review the number hidden nodes in the second layer
hidden_nodes_layer2

29

In [255]:
# Create the Sequential model instance
nn = Sequential()

In [256]:
# Add the first hidden layer
nn.add(Dense(units=hidden_nodes_layer1, activation='relu', input_dim=number_input_features))

In [257]:
# Add the second hidden layer
nn.add(Dense(units=hidden_nodes_layer2, activation='relu'))

In [258]:
# Add the output layer to the model specifying the number of output neurons and activation function
nn.add(Dense(units=number_output_neurons, activation='sigmoid'))

In [259]:
# Display the Sequential model summary
nn.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 58)                6786      
                                                                 
 dense_31 (Dense)            (None, 29)                1711      
                                                                 
 dense_32 (Dense)            (None, 1)                 30        
                                                                 
Total params: 8,527
Trainable params: 8,527
Non-trainable params: 0
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [260]:
# Compile the Sequential model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [261]:
# Fit the model using 50 epochs and the training data
fit_model = nn.fit(X_train_scaled, y_train, epochs=50, verbose=1)

Epoch 1/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5750 - accuracy: 0.7194
Epoch 2/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5541 - accuracy: 0.7301
Epoch 3/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5516 - accuracy: 0.7298
Epoch 4/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5488 - accuracy: 0.7325
Epoch 5/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5475 - accuracy: 0.7312
Epoch 6/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5465 - accuracy: 0.7333
Epoch 7/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5459 - accuracy: 0.7330
Epoch 8/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5440 - accuracy: 0.7338
Epoch 9/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5429 - accuracy: 0.7349
Epoch 10/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5424 - accuracy: 0.7344

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [262]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5549 - accuracy: 0.7326 - 383ms/epoch - 1ms/step
Loss: 0.5548954606056213, Accuracy: 0.7325947284698486


### Step 4: Save and export your model to an HDF5 file, and name the file `AlphabetSoup.h5`. 


In [263]:
# Set the model's file path
file_path = "Resources/model.h5"

# Export your model to a HDF5 file
nn.save(file_path)

---

## Optimize the neural network model


### Step 1: Define at least three new deep neural network models (resulting in the original plus 3 optimization attempts). With each, try to improve on your first model’s predictive accuracy.

> **Rewind** Recall that perfect accuracy has a value of 1, so accuracy improves as its value moves closer to 1. To optimize your model for a predictive accuracy as close to 1 as possible, you can use any or all of the following techniques:
> * Adjust the input data by dropping different features columns to ensure that no variables or outliers confuse the model.
> * Add more neurons (nodes) to a hidden layer.
> * Add more hidden layers.
> * Use different activation functions for the hidden layers.
> * Add to or reduce the number of epochs in the training regimen.

### Alternative Model 1

In [264]:
# drop extra features, so as not to fall into the Dummy Variable Trap
# Create a OneHotEncoder instance
enc_A1 = OneHotEncoder(sparse=False, drop='first')

# Encode the categorcal variables using OneHotEncoder
encoded_A1 = enc_A1.fit_transform(applicant_data_df[categorical_variables])

# Create a DataFrame with the encoded variables
encoded_df_A1 = pd.DataFrame(encoded_A1, columns=enc_A1.get_feature_names(categorical_variables))

# Review the DataFrame
print(f"Shape: {encoded_df_A1.shape}\n")
encoded_df_A1.head()

Shape: (34299, 107)



,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,APPLICATION_TYPE_T9,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C1230,CLASSIFICATION_C1234,CLASSIFICATION_C1235,CLASSIFICATION_C1236,CLASSIFICATION_C1237,CLASSIFICATION_C1238,CLASSIFICATION_C1240,CLASSIFICATION_C1245,CLASSIFICATION_C1246,CLASSIFICATION_C1248,CLASSIFICATION_C1250,CLASSIFICATION_C1256,CLASSIFICATION_C1257,CLASSIFICATION_C1260,CLASSIFICATION_C1267,CLASSIFICATION_C1270,CLASSIFICATION_C1278,CLASSIFICATION_C1280,CLASSIFICATION_C1283,CLASSIFICATION_C1300,CLASSIFICATION_C1370,CLASSIFICATION_C1400,CLASSIFICATION_C1500,CLASSIFICATION_C1570,CLASSIFICATION_C1580,CLASSIFICATION_C1600,CLASSIFICATION_C1700,CLASSIFICATION_C1720,CLASSIFICATION_C1728,CLASSIFICATION_C1732,CLASSIFICATION_C1800,CLASSIFICATION_C1820,CLASSIFICATION_C1900,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C2150,CLASSIFICATION_C2170,CLASSIFICATION_C2190,CLASSIFICATION_C2300,CLASSIFICATION_C2380,CLASSIFICATION_C2400,CLASSIFICATION_C2500,CLASSIFICATION_C2561,CLASSIFICATION_C2570,CLASSIFICATION_C2600,CLASSIFICATION_C2700,CLASSIFICATION_C2710,CLASSIFICATION_C2800,CLASSIFICATION_C3000,CLASSIFICATION_C3200,CLASSIFICATION_C3700,CLASSIFICATION_C4000,CLASSIFICATION_C4100,CLASSIFICATION_C4120,CLASSIFICATION_C4200,CLASSIFICATION_C4500,CLASSIFICATION_C5000,CLASSIFICATION_C5200,CLASSIFICATION_C6000,CLASSIFICATION_C6100,CLASSIFICATION_C7000,CLASSIFICATION_C7100,CLASSIFICATION_C7120,CLASSIFICATION_C7200,CLASSIFICATION_C7210,CLASSIFICATION_C8000,CLASSIFICATION_C8200,CLASSIFICATION_C8210,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_Y
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0

In [265]:
# Add the numerical variables from the original DataFrame to the one-hot encoding DataFrame
numerical_variables_df = applicant_data_df.select_dtypes(include='int64')
processed_df_drop_first = pd.concat([numerical_variables_df, encoded_df_A1], axis=1)

# Review the Dataframe
print(f"Shape: {processed_df_drop_first.shape}\n")
print(f"dtypes:\n{processed_df_drop_first.dtypes}")

Shape: (34299, 110)

dtypes:
STATUS                        int64
ASK_AMT                       int64
IS_SUCCESSFUL                 int64
APPLICATION_TYPE_T12        float64
APPLICATION_TYPE_T13        float64
                             ...   
INCOME_AMT_1M-5M            float64
INCOME_AMT_25000-99999      float64
INCOME_AMT_50M+             float64
INCOME_AMT_5M-10M           float64
SPECIAL_CONSIDERATIONS_Y    float64
Length: 110, dtype: object


In [267]:
# Define the target set y using the IS_SUCCESSFUL column
y_A1 = processed_df_drop_first['IS_SUCCESSFUL']

# Define features set X by selecting all columns but IS_SUCCESSFUL
X_A1 = processed_df_drop_first.drop(columns=['IS_SUCCESSFUL'])

# Review the features DataFrame
print(f"Shape: {X_A1.shape}\n")

# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train_A1, X_test_A1, y_train_A1, y_test_A1 = train_test_split(X_A1, y_A1, random_state=1)

# Create a StandardScaler instance
scaler_A1 = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler_A1 = scaler_A1.fit(X_train_A1)

# Fit the scaler to the features training dataset
X_train_scaled_A1 = X_scaler_A1.transform(X_train_A1)
X_test_scaled_A1 = X_scaler_A1.transform(X_test_A1)

Shape: (34299, 109)



In [268]:
# Define the the number of inputs (features) to the model
number_input_features_A1 = len(X_train_A1.iloc[0])

# Review the number of features
number_input_features_A1

109

In [269]:
# Define the number of neurons in the output layer
number_output_neurons_A1 = 1

# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A1 = (number_input_features_A1 + 1) // 2

# Review the number of hidden nodes in the first layer
hidden_nodes_layer1_A1

55

In [270]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2_A1 = (hidden_nodes_layer1_A1 + 1) // 2
hidden_nodes_layer2_A1

28

In [271]:
# Create the Sequential model instance
nn_A1 = Sequential()

In [272]:
# First hidden layer
nn_A1.add(Dense(units=hidden_nodes_layer1_A1, activation='relu', input_dim=number_input_features_A1))

# Add the second hidden layer
nn_A1.add(Dense(units=hidden_nodes_layer2_A1, activation='relu'))

# Output layer
# Add the output layer to the model specifying the number of output neurons and activation function
nn_A1.add(Dense(units=number_output_neurons_A1, activation='sigmoid'))

# Check the structure of the model
nn_A1.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_33 (Dense)            (None, 55)                6050      
                                                                 
 dense_34 (Dense)            (None, 28)                1568      
                                                                 
 dense_35 (Dense)            (None, 1)                 29        
                                                                 
Total params: 7,647
Trainable params: 7,647
Non-trainable params: 0
_________________________________________________________________


In [273]:
# Compile the Sequential model
nn_A1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [274]:
# Fit the model using 50 epochs and the training data
fit_model_A1 = nn_A1.fit(X_train_scaled_A1, y_train_A1, epochs=50)

Epoch 1/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5861 - accuracy: 0.7104
Epoch 2/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5550 - accuracy: 0.7289
Epoch 3/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5513 - accuracy: 0.7307
Epoch 4/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5494 - accuracy: 0.7314
Epoch 5/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5474 - accuracy: 0.7332
Epoch 6/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5460 - accuracy: 0.7329
Epoch 7/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5447 - accuracy: 0.7339
Epoch 8/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5448 - accuracy: 0.7334
Epoch 9/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5442 - accuracy: 0.7341
Epoch 10/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5429 - accuracy: 0.7339

#### Alternative Model 2

Okay, so Alternative 1 performed slightly better than the original model, so now I'm going
to try going **3** networks deep.

In [275]:
# Define the the number of inputs (features) to the model
number_input_features_A2 = len(X_train_A1.iloc[0])

# Review the number of features
number_input_features_A2

109

In [276]:
# Define the number of neurons in the output layer
number_output_neurons_A2 = 1

# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A2 = (number_input_features_A2 + 1) // 2
hidden_nodes_layer2_A2 = (hidden_nodes_layer1_A2 + 1) // 2
hidden_nodes_layer3_A2 = (hidden_nodes_layer2_A2 + 1) // 2
hidden_nodes_layer3_A2

14

In [277]:
# Create the Sequential model instance
nn_A2 = Sequential()

In [278]:
# First hidden layer
nn_A2.add(Dense(units=hidden_nodes_layer1_A2, activation='relu', input_dim=number_input_features_A2))

# Add the second hidden layer
nn_A2.add(Dense(units=hidden_nodes_layer2_A2, activation='relu'))

# Add the third hidden layer
nn_A2.add(Dense(units=hidden_nodes_layer3_A2, activation='relu'))

# Output layer
# Add the output layer to the model specifying the number of output neurons and activation function
nn_A2.add(Dense(units=number_output_neurons_A2, activation='sigmoid'))

# Check the structure of the model
nn_A2.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_36 (Dense)            (None, 55)                6050      
                                                                 
 dense_37 (Dense)            (None, 28)                1568      
                                                                 
 dense_38 (Dense)            (None, 14)                406       
                                                                 
 dense_39 (Dense)            (None, 1)                 15        
                                                                 
Total params: 8,039
Trainable params: 8,039
Non-trainable params: 0
_________________________________________________________________


In [279]:
# Compile the model
nn_A2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [280]:
# Fit the model
fit_model_A2 = nn_A2.fit(X_train_scaled_A1, y_train_A1, epochs=50)

Epoch 1/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5803 - accuracy: 0.7156
Epoch 2/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5549 - accuracy: 0.7265
Epoch 3/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5504 - accuracy: 0.7286
Epoch 4/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5479 - accuracy: 0.7293
Epoch 5/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5469 - accuracy: 0.7305
Epoch 6/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5453 - accuracy: 0.7322
Epoch 7/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5449 - accuracy: 0.7318: 0s - l
Epoch 8/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5442 - accuracy: 0.7322
Epoch 9/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5439 - accuracy: 0.7333
Epoch 10/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5436 - accuracy

### Alternative Model 3

The instructions said to make three alternative models, but there was only a framework for two 
alternative models.  I'm a little obsessed with trying to get a more accurate model, so will try
one more time.

I made plots of all the variables and their rate of IS_SUCCESSFUL and removed the encoded variables
that do not seem to have much of a correlation with the model.

In [287]:
# reset the OneHotEncoder instance
enc_A3 = OneHotEncoder(sparse=False)

# Encode the categorcal variables using OneHotEncoder
encoded_A3 = enc_A3.fit_transform(applicant_data_df[categorical_variables])

# Create a DataFrame with the encoded variables
encoded_df_A3 = pd.DataFrame(encoded_A3, columns=enc_A3.get_feature_names(categorical_variables))

# Review the DataFrame
print(f"Shape: {encoded_df_A3.shape}\n")
encoded_df_A3.head()

Shape: (34299, 114)



,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,APPLICATION_TYPE_T9,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,CLASSIFICATION_C0,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C1230,CLASSIFICATION_C1234,CLASSIFICATION_C1235,CLASSIFICATION_C1236,CLASSIFICATION_C1237,CLASSIFICATION_C1238,CLASSIFICATION_C1240,CLASSIFICATION_C1245,CLASSIFICATION_C1246,CLASSIFICATION_C1248,CLASSIFICATION_C1250,CLASSIFICATION_C1256,CLASSIFICATION_C1257,CLASSIFICATION_C1260,CLASSIFICATION_C1267,CLASSIFICATION_C1270,CLASSIFICATION_C1278,CLASSIFICATION_C1280,CLASSIFICATION_C1283,CLASSIFICATION_C1300,CLASSIFICATION_C1370,CLASSIFICATION_C1400,CLASSIFICATION_C1500,CLASSIFICATION_C1570,CLASSIFICATION_C1580,CLASSIFICATION_C1600,CLASSIFICATION_C1700,CLASSIFICATION_C1720,CLASSIFICATION_C1728,CLASSIFICATION_C1732,CLASSIFICATION_C1800,CLASSIFICATION_C1820,CLASSIFICATION_C1900,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C2150,CLASSIFICATION_C2170,CLASSIFICATION_C2190,CLASSIFICATION_C2300,CLASSIFICATION_C2380,CLASSIFICATION_C2400,CLASSIFICATION_C2500,CLASSIFICATION_C2561,CLASSIFICATION_C2570,CLASSIFICATION_C2600,CLASSIFICATION_C2700,CLASSIFICATION_C2710,CLASSIFICATION_C2800,CLASSIFICATION_C3000,CLASSIFICATION_C3200,CLASSIFICATION_C3700,CLASSIFICATION_C4000,CLASSIFICATION_C4100,CLASSIFICATION_C4120,CLASSIFICATION_C4200,CLASSIFICATION_C4500,CLASSIFICATION_C5000,CLASSIFICATION_C5200,CLASSIFICATION_C6000,CLASSIFICATION_C6100,CLASSIFICATION_C7000,CLASSIFICATION_C7100,CLASSIFICATION_C7120,CLASSIFICATION_C7200,CLASSIFICATION_C7210,CLASSIFICATION_C8000,CLASSIFICATION_C8200,CLASSIFICATION_C8210,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [288]:
# Add the numerical variables from the original DataFrame to the one-hot encoding DataFrame
numerical_variables_df = applicant_data_df.select_dtypes(include='int64')
df_reduced_A3 = pd.concat([numerical_variables_df, encoded_df_A3], axis=1)

# Review the Dataframe
print(f"Shape: {df_reduced.shape}\n")
print(f"dtypes:\n{df_reduced.dtypes}")

Shape: (34299, 117)

dtypes:
STATUS                        int64
ASK_AMT                       int64
IS_SUCCESSFUL                 int64
APPLICATION_TYPE_T10        float64
APPLICATION_TYPE_T12        float64
                             ...   
INCOME_AMT_25000-99999      float64
INCOME_AMT_50M+             float64
INCOME_AMT_5M-10M           float64
SPECIAL_CONSIDERATIONS_N    float64
SPECIAL_CONSIDERATIONS_Y    float64
Length: 117, dtype: object


In [289]:
# Define the target set y using the IS_SUCCESSFUL column
y_A3 = df_reduced_A3['IS_SUCCESSFUL']

# Define features set X by reducing down to the features that might have a linear relationship
X_A3 = df_reduced_A3[['ASK_AMT', 
    'APPLICATION_TYPE_T3', 'APPLICATION_TYPE_T4',
    'APPLICATION_TYPE_T6', 'APPLICATION_TYPE_T5', 'APPLICATION_TYPE_T19',
    'APPLICATION_TYPE_T8', 'APPLICATION_TYPE_T7', 'APPLICATION_TYPE_T10',
    'AFFILIATION_Independent', 'AFFILIATION_CompanySponsored', 
    'CLASSIFICATION_C1000',
    'CLASSIFICATION_C1200',
    'CLASSIFICATION_C2000',
    'CLASSIFICATION_C2100',
    'CLASSIFICATION_C3000',
    'CLASSIFICATION_C7000',
    'USE_CASE_Preservation', 'USE_CASE_ProductDev',
    'ORGANIZATION_Association', 'ORGANIZATION_Co-operative',
    'ORGANIZATION_Trust']]

In [290]:
# Being DRY, so defining a function to rescale the 'ASK_AMT' so it is not so imbalanced
def Ask_Group (x):
    if x == 5000:
        y = 0
    elif x >= 1000000:
        y = 1
    elif x >= 100000:
        y = .75
    elif x > 5000:
        y = .25
    else: 
        y = -1
     
    return y

# calling the Ask_Group function to make four groups
X_A3['ASK_AMT_GROUP'] = X_A3['ASK_AMT'].apply(Ask_Group)

# Drop the 'ASK_AMT' column now
X_A3.drop(columns=['ASK_AMT'], inplace=True)

# Review the features DataFrame
print(f"Shape: {X_A3.shape}\n")
X_A3.head(10)

Shape: (34299, 22)



<ipython-input-290-b52ef9114ea2>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_A3['ASK_AMT_GROUP'] = X_A3['ASK_AMT'].apply(Ask_Group)
C:\Users\Dell\anaconda3\envs\dev\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T6,APPLICATION_TYPE_T5,APPLICATION_TYPE_T19,APPLICATION_TYPE_T8,APPLICATION_TYPE_T7,APPLICATION_TYPE_T10,AFFILIATION_Independent,AFFILIATION_CompanySponsored,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C3000,CLASSIFICATION_C7000,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Trust,ASK_AMT_GROUP
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.00
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.75
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.00
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.25
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.75
5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.00
6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.25
7,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.00
8,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.25
9,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.00


In [291]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train_A3, X_test_A3, y_train_A3, y_test_A3 = train_test_split(X_A3, y_A3, random_state=1)

# I ran this model with and without a scaler and it performs better without scaling

In [292]:
# Define the the number of inputs (features) to the model
number_input_features_A3 = len(X_train_A3.iloc[0])

# Review the number of features
number_input_features_A3

22

In [293]:
# Define the number of neurons in the output layer
number_output_neurons_A3 = 1

# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A3 = (number_input_features_A3 + 1) // 2

# Review the number of hidden nodes in the first layer
hidden_nodes_layer1_A3

11

In [294]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2_A3 = (hidden_nodes_layer1_A3 + 1) // 2
hidden_nodes_layer2_A3

6

In [295]:
# Create the Sequential model instance
nn_A3 = Sequential()

In [296]:
### Compile the Sequential model
nn_A3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [297]:
# Fit the model
fit_model_A3 = nn_A3.fit(X_train_A3, y_train_A3, epochs=50)

Epoch 1/50
804/804 [==============================] - 2s 2ms/step - loss: 7.9306 - accuracy: 0.3820
Epoch 2/50
804/804 [==============================] - 1s 1ms/step - loss: 7.9306 - accuracy: 0.3820: 0s - loss: 7.9527 - accura
Epoch 3/50
804/804 [==============================] - 1s 1ms/step - loss: 7.9306 - accuracy: 0.3820
Epoch 4/50
804/804 [==============================] - 1s 1ms/step - loss: 7.9306 - accuracy: 0.3820
Epoch 5/50
804/804 [==============================] - 1s 1ms/step - loss: 7.9306 - accuracy: 0.3820
Epoch 6/50
804/804 [==============================] - 1s 1ms/step - loss: 7.9307 - accuracy: 0.3820
Epoch 7/50
804/804 [==============================] - 1s 1ms/step - loss: 7.9306 - accuracy: 0.3820
Epoch 8/50
804/804 [==============================] - 1s 1ms/step - loss: 7.9306 - accuracy: 0.3820
Epoch 9/50
804/804 [==============================] - 1s 1ms/step - loss: 7.9306 - accuracy: 0.3820
Epoch 10/50
804/804 [==============================] - 1s 1ms/step - los

### Step 2: After finishing your models, display the accuracy scores achieved by each model, and compare the results.

In [281]:
print("Original Model Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Original Model Results
268/268 - 0s - loss: 0.5549 - accuracy: 0.7326 - 338ms/epoch - 1ms/step
Loss: 0.5548954606056213, Accuracy: 0.7325947284698486


In [284]:
print("Alternative Model 1 Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_A1.evaluate(X_test_scaled_A1, y_test_A1, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Alternative Model 1 Results
268/268 - 0s - loss: 0.5592 - accuracy: 0.7304 - 325ms/epoch - 1ms/step
Loss: 0.5592098832130432, Accuracy: 0.7303789854049683


In [285]:
print("Alternative Model 2 Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_A2.evaluate(X_test_scaled_A1, y_test_A1, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Alternative Model 2 Results
268/268 - 0s - loss: 0.5577 - accuracy: 0.7261 - 416ms/epoch - 2ms/step
Loss: 0.5577303767204285, Accuracy: 0.726064145565033


In [299]:
print("Alternative Model 3 Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_A3.evaluate(X_test_A3, y_test_A3, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Alternative Model 3 Results
268/268 - 0s - loss: 7.8927 - accuracy: 0.3840 - 394ms/epoch - 1ms/step
Loss: 7.892729759216309, Accuracy: 0.3840233087539673


### Step 3: Save each of your alternative models as an HDF5 file.


In [300]:
# Set the file path for the first alternative model
file_path = "Resources/model_A1.h5"

# Export your model to a HDF5 file
nn_A1.save(file_path)

In [301]:
# Set the file path for the second alternative model
file_path = "Resources/model_A2.h5"

# Export your model to a HDF5 file
nn_A2.save(file_path)

In [302]:
# Set the file path for the third alternative model
file_path = "Resources/model_A3.h5"

# Export your model to a HDF5 file
nn_A3.save(file_path)